In [2]:
import numpy as np
import pandas as pd
import duckdb

In [3]:
FILE_PATH = "./part4.csv"

In [4]:
df = pd.read_csv(FILE_PATH)

### I. Define and justify 5 core metrics relevant for the City of Chicago government tracking vendor contracts. 


#### 1. Number of contract amendment

A contract amendment is a modification to the original contract. That affects:
- Extending the contract duration.
- Increasing or decreasing the budget
- Changing the scope of work.
- Fixing errors or adding new clauses.


If Contracts with many amendments could reflect:
- The original scope was unclear.
- The project was poorly scoped or estimated.
- Requirements changed frequently.


So if we define with contract have high number of amendments ==> We need take necessary actions.

In this analysis, we categorize contract into 3 classes:
- `Low risk`: < 50.
- `Moderate risk`: >= 50 and < 100.
- `High risk`: >= 100.

In [6]:
# Run SQL to count amendments per contract
sql: str = """
WITH amendment_count AS (
  SELECT
    "Purchase Order (Contract) Number",
    COUNT(*) AS amendment_count,
    SUM("Award Amount") AS total_award_amount
  FROM
    df
  GROUP BY
    "Purchase Order (Contract) Number"
)
SELECT
  "Purchase Order (Contract) Number",
  CASE
    WHEN
      amendment_count < 50
    THEN
      'low'
    WHEN
      amendment_count >= 50
      AND amendment_count < 100
    THEN
      'Moderate'
    ELSE
      'High'
  END AS "risk",
  amendment_count,
  total_award_amount
FROM
  amendment_count
ORDER BY
  amendment_count DESC
"""
duckdb.query(sql).fetchdf()

,Purchase Order (Contract) Number,risk,amendment_count,total_award_amount
0,P011437,High,352,254910056.3
1,19486,High,216,150571307.5
2,P011887,High,180,87709288.1
3,129033,High,177,213393000.5
4,24640,High,173,170207806.5
...,...,...,...,...
86437,23395,low,1,20000.0
86438,269168,low,1,40302.0
86439,127286,low,1,55000.0
86440,97213,low,1,86581.0


In [ ]:
# Run SQL to count amendments per contract
sql: str = """
SELECT
  "Purchase Order (Contract) Number",
  MIN("Start Date") AS "Start Date",
  MAX("End Date") AS "End Date"
FROM
  df
GROUP BY
  "Purchase Order (Contract) Number"
ORDER BY
  sum_duration DESC
"""
duckdb.query(sql).fetchdf()

,Purchase Order (Contract) Number,sum_duration,Start Date,End Date
0,P011437,345660.0,1998-07-17 00:00:00,2004-03-19 00:00:00
1,T26824,259811.0,1992-12-15 00:00:00,2025-12-31 00:00:00
2,T25068,205932.0,1995-09-11 00:00:00,2007-12-31 00:00:00
3,6078,182631.0,1990-01-01 00:00:00,2027-01-31 00:00:00
4,6082,156697.0,1994-01-01 00:00:00,2027-01-31 00:00:00
...,...,...,...,...
86437,272190,NaN,None,None
86438,184677,NaN,None,None
86439,S026706,NaN,None,None
86440,D2990251985,NaN,None,None


#### 2. Contract Duration (Days)

The Contract Duration (Days) feature gives us a very important insight into the length of time each contract is expected to be active. This helps assess planning, project scope, risk, and resource commitment. 

In [48]:
# During the investigation, we recognized some weird records when "Start Date" > "End Date" (115 records).
# At this moment, we decided to removed all weird records in 2nd metric
sql: str = """
SELECT
  *
FROM
  df
WHERE
    "Start Date" <= "End Date"
"""
df_case2: pd.DataFrame = duckdb.query(sql).fetchdf()

In [49]:
# Run SQL to count amendments per contract
sql: str = """
  SELECT
    "Purchase Order (Contract) Number",
    "Department",
    DATE_DIFF('day', MIN("Start Date"::DATE), MAX("End Date"::DATE)) AS contract_duration_day
  FROM
    df_case2
  GROUP BY
    "Purchase Order (Contract) Number",
    "Department"
  HAVING
    contract_duration_day IS NOT NULL
  ORDER BY
    1, 2
"""
duckdb.query(sql).fetchdf()

,Purchase Order (Contract) Number,Department,contract_duration_day
0,000600049,DEPT OF BUSINESS & INFORMATION SERVICES,365
1,000600988,DEPT OF BUSINESS & INFORMATION SERVICES,1096
2,002100928,DEPARTMENT OF HOUSING,30
3,003500301,DEPARTMENT OF PROCUREMENT SERVICES,807
4,004800007,MAYORS OFFICE FOR PEOPLE WITH DISABILITIES,291
...,...,...,...
70830,V3002R,DEPT OF GENERAL SERVICES,365
70831,V5006,DEPT OF GENERAL SERVICES,365
70832,V5006AR1,DEPT OF GENERAL SERVICES,59
70833,V6002,DEPT OF GENERAL SERVICES,549


Based on that query we can analyze what contract took long time, it reflect the performance of Department, excessive resources, ...

And we can detect which departments are slow (poor) by calculating the average time across their entire contracts.

In [50]:
# Run SQL to count amendments per contract
sql: str = """
    WITH raw_compute AS (
        SELECT
            "Purchase Order (Contract) Number",
            "Department",
            DATE_DIFF('day', MIN("Start Date"::DATE), MAX("End Date"::DATE)) AS contract_duration_day
        FROM
            df_case2
        GROUP BY
            "Purchase Order (Contract) Number",
            "Department"
        HAVING
            contract_duration_day IS NOT NULL
    )
    SELECT
        "Department",
        AVG(contract_duration_day) AS avg_contract_duration_day,
        COUNT(DISTINCT "Purchase Order (Contract) Number") AS number_of_contract,
        MIN(contract_duration_day) AS shortest_duration,
        MAX(contract_duration_day) AS longest_duration,
    FROM
        raw_compute
    GROUP BY
        "Department"
    ORDER BY
        2 DESC
"""
duckdb.query(sql).fetchdf()

,Department,avg_contract_duration_day,number_of_contract,shortest_duration,longest_duration
0,DEPARTMENT OF SPECIAL EVENTS,4928.000000,1,4928,4928
1,O'HARE MODERNIZATION PROGRAM,4200.000000,2,731,7669
2,DEPT OF LAW,3390.083333,12,2555,3834
3,DEPT OF INNOVATION & TECHNOLOGY,2722.000000,30,902,4931
4,DEPT OF ASSETS INFORMATION AND SERVICES,2460.994898,196,81,6232
...,...,...,...,...,...
66,LICENSE APPEAL COMMISSION,365.000000,1,365,365
67,INDEPENDENT POLICE REVIEW AUTHORITY,364.000000,1,364,364
68,DEPT OF SENIOR SERVICES,364.000000,1,364,364
69,DEPARTMENT OF BUILDINGS,307.360441,8792,0,8796


#### 3. Total spend per vendor

Tracks major suppliers, budget burden, diversity goals. Identifies which vendors receive the most funding from the City of Chicago.

In [68]:
# Run SQL to count amendments per contract
sql: str = """
    SELECT
        "Vendor Name",
        SUM("Award Amount")::FLOAT AS sum_award_amount,
        COUNT(DISTINCT "Purchase Order (Contract) Number") AS number_of_contract
    FROM
        df
    GROUP BY
        "Vendor Name"
    ORDER BY
        2 DESC,
        3 DESC

"""
duckdb.query(sql).fetchdf()

,Vendor Name,sum_award_amount,number_of_contract
0,BLUE CROSS & BLUE SHIELD,1.248646e+10,5
1,CAREMARK INC,2.636069e+09,2
2,BOARD OF EDUCATION OF THE CITY OF CHICAGO,1.856844e+09,280
3,PUBLIC BUILDING COMMISSION CHICAGO,1.840028e+09,30
4,ANN & ROBERT H. LURIE CHILDREN'S HOSPITAL OF C...,1.733073e+09,79
...,...,...,...
13252,CARSON'S TIRE SERVICE,0.000000e+00,1
13253,LA BOMBA RESTAURANT,0.000000e+00,1
13254,Montclare Dental Inc,0.000000e+00,1
13255,ECHO SUPPLY,-1.000000e-01,1


#### 4. DUR Contract Share

DUR stands for "Depends Upon Requirements". a type of contract where:

- The city does not commit to a specific quantity or total cost.

- Services or goods are purchased as needed during the contract period.

- The Award Amount is often $0, because spending depends on future demand.


DUR Contract Share =
The percentage of all contracts that are DUR-type (i.e., open-ended, $0 contracts).

In [ ]:
# Run SQL to count amendments per contract
sql: str = """
    SELECT 
    ROUND(
      100.0 * SUM(CASE WHEN "Is Blanket Contract" = TRUE THEN 1 ELSE 0 END) / COUNT(*), 
      2
    ) AS dur_contract_share_percent
  FROM df;
"""
duckdb.query(sql).fetchdf()

,dur_contract_share_percent
0,34.87


Based on the output, Approximately 1 in 3 contracts issued by the City of Chicago are Blanket (DUR) contracts — open-ended agreements that depend on future requirements.

While this offers procurement flexibility, it also highlights the need for rigorous monitoring of amendments and post-award expenditures to ensure fiscal accountability.

#### 5. Average Award Amount per Department

Reveals which departments award the highest-value contracts on average — helps in budget analysis

In [66]:
# Run SQL to count amendments per contract
sql: str = """
    SELECT 
        Department,
        ROUND(AVG("Award Amount"), 2) AS avg_award_amount,
        COUNT(*) AS contract_count
    FROM df
        GROUP BY Department
    ORDER BY 
        2 DESC;
"""
duckdb.query(sql).fetchdf()

,Department,avg_award_amount,contract_count
0,FINANCE GENERAL,16500000.00,2
1,DEPARTMENT OF FLEET AND FACILITY MANAGEMENT,11868141.18,255
2,DEPARTMENT OF FINANCE,11686868.33,1578
3,DEPT OF ASSETS INFORMATION AND SERVICES,7114956.50,335
4,DEPARTMENT OF TECHNOLOGY AND INNOVATION,6688083.31,94
...,...,...,...
68,LICENSE APPEAL COMMISSION,512.00,1
69,BOARD OF ELECTION COMMISSIONERS,0.00,1
70,DEPT OF LAW,0.00,29
71,DEPT OF SENIOR SERVICES,0.00,1


### II. Design a reporting dashboard outline:

#### Intended Audience

Our target audience is government team as Budget & Finance Teams.

#### Core Charts and Visuals

Overview KPI: Use Metric Cards to visualize metrics as Total Awarded Amount, Total Contracts, Total Vendors, % Blanket Contracts (DUR).

Top vendor: Bar Chart

Amendments Tracker: Table + Bar Chart

Contract Duration: Histogram + Average Line

Time Trend:Line Chart - Award Amount over time (monthly/yearly)

Map: Geo heatmap based on Zip / City / State

#### Filters / Slicers

Date Range (based on Approval Date or Start Date)

Department

Contract Type 

Vendor Location (City, State, Zip)

Has Amendment / Revision Count > 0

Is Blanket Contract

### III. Sample visualization

I prepared some sample dashboard to visulize that dataset.

Please run
```bash
streamlit run part5_visualization.py  
```

Then open http://localhost:8501